In [1]:
import pandas as pd
import folium
import geopandas as gpd
import folium.plugins as plugins

In [2]:
df = pd.read_csv('msft_clean_df.csv')

In [3]:
df.head()

,date,ip,loc,lat,long,Date,Day
0,2021-06-23 16:50:03 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 16:50:03+00:00,2021-06-23
1,2021-06-23 19:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 19:19:12+00:00,2021-06-23
2,2021-06-23 21:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 21:19:12+00:00,2021-06-23
3,2021-06-23 23:41:19 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 23:41:19+00:00,2021-06-23
4,2021-06-24 02:10:18 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-24 02:10:18+00:00,2021-06-24


In [4]:
df.columns

Index(['date', 'ip', 'loc', 'lat', 'long', 'Date', 'Day'], dtype='object')

In [5]:
df.dtypes

date     object
ip       object
loc      object
lat     float64
long    float64
Date     object
Day      object
dtype: object

In [6]:
# df["lat_long"] = list(zip(df["lat"], df["long"]))

In [7]:
df.head()

,date,ip,loc,lat,long,Date,Day
0,2021-06-23 16:50:03 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 16:50:03+00:00,2021-06-23
1,2021-06-23 19:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 19:19:12+00:00,2021-06-23
2,2021-06-23 21:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 21:19:12+00:00,2021-06-23
3,2021-06-23 23:41:19 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 23:41:19+00:00,2021-06-23
4,2021-06-24 02:10:18 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-24 02:10:18+00:00,2021-06-24


In [8]:
# lat_long = df['lat_long']

In [9]:
# dates = pd.to_datetime(df['Date'])

# function to generate base map, has default values for zoom and tiles
def generateBaseMap(loc, zoom=12, min_zoom=0, max_zoom=18, zoom_start=10, tiles='Stamen Toner', crs='ESPG2263'):
    '''
    Function that generates a Folium base map
    Input location lat/long
    Zoom level default 12
    Tiles default to Stamen Toner
    CRS default 2263 for NYC
    '''
    return folium.Map(location=loc, 
                      control_scale=True, 
                      zoom_start=zoom,
                      tiles=tiles)
  
mi = [42.279594, -83.732124] # generic nyc lat/lon in list format
base_map = generateBaseMap(mi) # pass lat/lon to function
base_map

from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime


#create base map
# mi_base_map = folium.Map(location = [42.279594, -83.732124], control_scale = True, zoom_start = 10, tiles = 'Stamen Toner')

#create the heatmap from list
HeatMap(data = df['lat_long'], radius=12).add_to(base_map)

base_map.save('bike_station_HeatMap.html')

# call map 
base_map

In [10]:
lat_long_list = []
for i in df['Date'].unique():
    temp=[]
    for index, instance in df[df['Date'] == i].iterrows():
        temp.append([instance['lat'],instance['long']])
    lat_long_list.append(temp)

In [11]:
lat_long_list[0][0]

[42.2709, -83.7282]

In [12]:
type(lat_long_list[0][0])

list

In [13]:
df['Date']= pd.to_datetime(df['Date'])
#creating a time index
time_index = []
for i in df['Date'].unique():
    time_index.append(i)
#formatting the index
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

In [14]:
# time_index

In [15]:
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

#Choosing the map type 
m = folium.Map(location=[42.2808, -83.7430],zoom_start = 11,control_scale=True, tiles='openstreetmap',attr="Stadia.AlidadeSmoothDark")

#Plot it on the map
HeatMapWithTime(lat_long_list,radius=10,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(m)
# Display the map
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)
m

In [16]:
m_n = folium.Map(df[['lat', 'long']].mean().values.tolist())

for lat, lon in zip(df['lat'], df['long']):
    folium.Marker([lat, lon]).add_to(m_n)

In [17]:
sw = df[['lat', 'long']].min().values.tolist()
ne = df[['lat', 'long']].max().values.tolist()

m_n.fit_bounds([sw, ne]) 
m_n

In [18]:
from folium.plugins import MarkerCluster
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster, HeatMapWithTime
import math

In [19]:
m_2 = folium.Map(location=[42.2808, -83.7430], tiles='openstreetmap', zoom_start=5)

# Add points to the map
for idx, row in df.iterrows():
    Marker([row['lat'], row['long']]).add_to(m_2)

In [20]:
m_3 = folium.Map(location=[42.2808, -83.7430], tiles='openstreetmap', zoom_start=5)

# Add points to the map
mc = MarkerCluster()
for idx, row in df.iterrows():
    if not math.isnan(row['long']) and not math.isnan(row['lat']):
        mc.add_child(Marker([row['lat'], row['long']]))
m_3.add_child(mc)

# Adds tool to the top right
from folium.plugins import MeasureControl
m_3.add_child(MeasureControl())

# Display the map
m_3

## Foursquare API work

In [21]:
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim 
import requests

In [22]:
CLIENT_ID = 'JKHM1V3IJMGJTDH13VB2UKKYNLHN0N3XG4UENX0JEKBUZSMP' # your Foursquare ID
CLIENT_SECRET = 'CTHTUE1JSINGUNVXSDL0JGPO1CRKOE312RYYA21LEBKE5ZSA' # your Foursquare Secret
VERSION = '20220208'
LIMIT = 100

In [23]:
df.head()

,date,ip,loc,lat,long,Date,Day
0,2021-06-23 16:50:03 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 16:50:03+00:00,2021-06-23
1,2021-06-23 19:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 19:19:12+00:00,2021-06-23
2,2021-06-23 21:19:12 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 21:19:12+00:00,2021-06-23
3,2021-06-23 23:41:19 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-23 23:41:19+00:00,2021-06-23
4,2021-06-24 02:10:18 UTC,208.75.19.158,"Ann Arbor, Michigan",42.2709,-83.7282,2021-06-24 02:10:18+00:00,2021-06-24


In [24]:
ndf = df[['loc','lat','long']]

In [25]:
nndf = ndf.drop_duplicates()

In [50]:
# pd.options.display.float_format = "{:,.2f}".format
nndf

,loc,lat,long,long_lat
0,"Ann Arbor, Michigan",42.27,-83.73,"(42.2709, -83.7282)"
44,"Detroit, Michigan",37.75,-83.11,"(37.751, -83.1087)"
73,"Ann Arbor, Michigan",42.27,-83.71,"(42.2709, -83.7068)"
80,"Ann Arbor, Michigan",42.71,-83.71,"(42.7069, -83.7068)"
81,"Ann Arbor, Michigan",42.28,-83.71,"(42.2804, -83.7068)"
109,"Hamtramck, Michigan",42.41,-83.06,"(42.4122, -83.0583)"
110,"Detroit, Michigan",42.39,-83.15,"(42.3914, -83.15)"
154,"Astoria, New York",40.77,-73.93,"(40.7706, -73.9265)"
168,"None, New Jersey",40.71,-74.21,"(40.7064, -74.2111)"
233,"None, Tennessee",36.17,-86.78,"(36.1671, -86.7844)"


In [51]:
nndf["long_lat"] = list(zip(nndf["lat"], nndf["long"]))


<ipython-input-51-023427462055>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nndf["long_lat"] = list(zip(nndf["lat"], nndf["long"]))


In [52]:
nndf

,loc,lat,long,long_lat
0,"Ann Arbor, Michigan",42.27,-83.73,"(42.2709, -83.7282)"
44,"Detroit, Michigan",37.75,-83.11,"(37.751, -83.1087)"
73,"Ann Arbor, Michigan",42.27,-83.71,"(42.2709, -83.7068)"
80,"Ann Arbor, Michigan",42.71,-83.71,"(42.7069, -83.7068)"
81,"Ann Arbor, Michigan",42.28,-83.71,"(42.2804, -83.7068)"
109,"Hamtramck, Michigan",42.41,-83.06,"(42.4122, -83.0583)"
110,"Detroit, Michigan",42.39,-83.15,"(42.3914, -83.15)"
154,"Astoria, New York",40.77,-73.93,"(40.7706, -73.9265)"
168,"None, New Jersey",40.71,-74.21,"(40.7064, -74.2111)"
233,"None, Tennessee",36.17,-86.78,"(36.1671, -86.7844)"


In [72]:
concat_df = pd.DataFrame()

In [88]:
for index, row in nndf.iterrows():
    url = "https://api.foursquare.com/v3/places/nearby?ll={},{}&limit=20".format(round(row['lat'], 2),round(row['long'], 2))
#     print(url)
    headers = {
    "Accept": "application/json",
    "Authorization": "fsq3chVTJib0Z11IA8qFisvs8p7dkOJ6ky0WEbGTZ9FQPqc="
    }
    result = requests.request("GET", url, headers=headers).json()
    result_df = json_normalize(result['results'])
    result_df = result_df.rename(columns={"geocodes.main.latitude": "lat", "geocodes.main.longitude": "long", "location.formatted_address": "address"})
#     print(result_df)
    concat_df = concat_df.append(result_df)

<ipython-input-88-93558f7678ae>:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  result_df = json_normalize(result['results'])


In [89]:
#concat_df

In [93]:
new_map = folium.Map(location=[42.2808, -83.7430], zoom_start=15)
# add a red circle marker to represent each visited locations
for lat, long in zip(nndf.lat, nndf.long):
    folium.features.CircleMarker(
        [lat, long],
        radius=10,
        color='red',
        #popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(new_map)
# add all venues as blue circle markers
for lat, long, label in zip(concat_df.lat, concat_df.long, concat_df.name):
    folium.features.CircleMarker(
        [lat, long],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(new_map)

In [94]:
new_map